Work in Progress ... to visualize internal activations

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
import itertools
import json
import pickle
import random
import sys

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import model_from_json
from keras import regularizers, optimizers
from keras.utils import np_utils, to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

import torch
from torchvision.utils import make_grid

import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# Set path so that functions can be imported from the utils script
sys.path.insert(0, '../')
from preprocess import sample_timeseries
from visualize import visualize_rgb, record_batch
from constants import *
import datasets
import util
import models

In [ ]:
train_parser = util.get_train_parser()
args = train_parser.parse_args(['--model_name', 'fcn_crnn', 
                                '--dataset', 'full', 
                                '--num_classes', str(4),
                                '--country', 'ghana',
                                '--batch_size', str(5),
                                '--hidden_dims', str(32),
                                '--crnn_num_layers', str(1),
                                '--use_s1', str(False),
                                '--use_s2', str(True),
                                '--sample_w_clouds', str(True),
                                '--include_clouds', str(True),
                                '--include_doy', str(True),
                                '--bidirectional', str(False),
                                '--shuffle', str(False)])


In [ ]:
model_name = 'fcn_crnn'
fcn_crnn_model = models.get_model(**vars(args))
fcn_crnn_model.load_state_dict(torch.load(fcn_crnn_model_path))
fcn_crnn_model.eval()
  
batch_num = 0
while batch_num < len(test_fnames):
    for inputs, targets, cloudmasks in dl:
        preds = fcn_crnn_model(inputs)
        labels_grid, inputs_grid, targets_grid, preds_grid, predsmask_grid = record_batch(
                                                                           inputs, cloudmasks, targets, preds, #<< preds
                                                                           args.num_classes, split, vis_data, 
                                                                           vis, args.include_doy, args.use_s1, 
                                                                           args.use_s2, model_name, args.time_slice, 
                                                                           save=False, save_dir=None, 
                                                                           show_visdom=False, show_matplot=True)